In [17]:
import pandas as pd
import pymysql
import getpass
import statsmodels.api as sm

# Olist data

Estimating review_score

In [2]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=getpass.getpass(),
                       db="olist")

········


In [3]:
query = """
SELECT
    o.order_id,
    dor.review_score,
    DATEDIFF(DATE(o.order_estimated_delivery_date), DATE(o.order_purchase_timestamp)) AS estimated_delivery_days,
    DATEDIFF(DATE(o.order_delivered_customer_date), DATE(o.order_purchase_timestamp)) AS actual_delivery_days,
    DATEDIFF(DATE(o.order_estimated_delivery_date), DATE(o.order_purchase_timestamp)) -
        DATEDIFF(DATE(o.order_delivered_customer_date), DATE(o.order_purchase_timestamp)) AS delivery_diff,
    oid.total_price,
    oid.shipping_costs,
    oid.order_size,
    oid.has_product_photo,
    oid.has_product_decription
FROM orders o
    JOIN temp_dedup_order_reviews dor
    ON o.order_id = dor.order_id
    JOIN temp_order_item_details oid
    ON o.order_id = oid.order_id
WHERE o.order_status = 'delivered';
"""

df = pd.read_sql(query, conn)

In [4]:
df.head()

,order_id,review_score,estimated_delivery_days,actual_delivery_days,delivery_diff,total_price,shipping_costs,order_size,has_product_photo,has_product_decription
0,e481f51cbdc54678b7cc49136f2d6af7,4,16,8.0,8.0,29.990000,8.720000,1,1.0,1.0
1,53cdb2fc8bc7dce0b6741e2150273451,4,20,14.0,6.0,118.699997,22.760000,1,1.0,1.0
2,47770eb9100c2d0c44946d9cf07ec65d,5,27,9.0,18.0,159.899994,19.219999,1,1.0,1.0
3,949d5b44dbf5de918fe9c16f97b45f8a,5,27,14.0,13.0,45.000000,27.200001,1,1.0,1.0
4,ad21c59c0840e6cb83a9ceb5573f8159,5,13,3.0,10.0,19.900000,8.720000,1,1.0,1.0


In [5]:
df.dtypes

order_id                    object
review_score                 int64
estimated_delivery_days      int64
actual_delivery_days       float64
delivery_diff              float64
total_price                float64
shipping_costs             float64
order_size                   int64
has_product_photo          float64
has_product_decription     float64
dtype: object

In [9]:
# check distribution of order sizes
df["order_size"].value_counts(normalize=True)

1     0.900133
2     0.076619
3     0.013537
4     0.005131
5     0.002000
6     0.001980
7     0.000228
10    0.000083
8     0.000083
12    0.000052
11    0.000041
9     0.000031
20    0.000021
14    0.000021
15    0.000021
13    0.000010
21    0.000010
Name: order_size, dtype: float64

## 1. `review_score` as a function of `total_price`

In [10]:
df_1 = df.copy()

In [12]:
df_1 = df_1[["review_score", "total_price"]]

In [16]:
# 1. create a constant
df_1 = df_1.assign(const=1)
df_1.head()

,review_score,total_price,const
0,4,29.990000,1
1,4,118.699997,1
2,5,159.899994,1
3,5,45.000000,1
4,5,19.900000,1


In [21]:
lr_1 = sm.OLS(endog=df_1["review_score"],
              exog=df_1[["const", "total_price"]]).fit()

In [22]:
lr_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     126.3
Date:                Tue, 04 Feb 2020   Prob (F-statistic):           2.72e-29
Time:                        15:08:56   Log-Likelihood:            -1.6176e+05
No. Observations:               96478   AIC:                         3.235e+05
Df Residuals:                   96476   BIC:                         3.235e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.1754      0.005    838.238      0.000       4.166       4.185
total_price    -0.0002   1.99e-05    -11.240      0.000      -0.000      -0.000
==============================================================================
Omnibus:                    20874.764   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37202.579
Skew:                          -1.457   Prob(JB):                         0.00
Kurtosis:                       3.871   Cond. No.                         299.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

For every additional Foreal, our reviews decrease by 0.0002

## 2. `review_score` as a function of `total_price`, `shipping_costs`, `actual_delivery_days`

In [23]:
df_2 = df.copy()
df_2 = df_2[["review_score", "total_price", "shipping_costs", "actual_delivery_days"]]
df_2.head()

,review_score,total_price,shipping_costs,actual_delivery_days
0,4,29.990000,8.720000,8.0
1,4,118.699997,22.760000,14.0
2,5,159.899994,19.219999,9.0
3,5,45.000000,27.200001,14.0
4,5,19.900000,8.720000,3.0


In [24]:
lr_2 = sm.OLS(endog=df_2["review_score"],
              exog=df_2[["total_price", "shipping_costs", "actual_delivery_days"]],
              hasconst=True).fit()

MissingDataError: exog contains inf or nans

In [28]:
df_2.isna().sum()

review_score            0
total_price             0
shipping_costs          0
actual_delivery_days    8
dtype: int64

In [30]:
df_2["actual_delivery_days"][df_2["actual_delivery_days"].isna()]

2921    NaN
20021   NaN
42530   NaN
76909   NaN
80392   NaN
89883   NaN
94731   NaN
95117   NaN
Name: actual_delivery_days, dtype: float64

In [32]:
df_2[df_2["actual_delivery_days"].isna()]

,order_id,review_score,estimated_delivery_days,actual_delivery_days,delivery_diff,total_price,shipping_costs,order_size,has_product_photo,has_product_decription
2921,2d1e2d5bf4dc7227b3bfebb81328c15f,5,20,NaN,NaN,117.300003,17.530001,1,1.0,1.0
20021,f5dd62b788049ad9fc0526e3ad11a097,5,26,NaN,NaN,329.000000,25.240000,1,1.0,1.0
42530,2ebdfc4f15f23b91474edf87475f108e,5,29,NaN,NaN,139.000000,19.070000,1,1.0,1.0
76909,e69f75a717d64fc5ecdfae42b2e8e086,5,29,NaN,NaN,139.000000,19.070000,1,1.0,1.0
80392,0d3268bad9b086af767785e3f0fc0133,5,23,NaN,NaN,188.990005,15.630000,1,1.0,1.0
89883,2d858f451373b04fb5c984a1cc2defaf,5,29,NaN,NaN,179.000000,15.000000,1,1.0,1.0
94731,ab7c89dc1bf4a1ead9d6ec1ec8968a84,1,18,NaN,NaN,110.989998,9.130000,1,1.0,1.0
95117,20edc82cf5400ce95e1afacc25798b31,5,22,NaN,NaN,45.900002,9.070000,1,1.0,1.0


drop them

In [34]:
df_2 = df_2.dropna()

In [39]:
df_2 = df_2.assign(const=1)

In [42]:
lr_2 = sm.OLS(endog=df_2["review_score"],
              exog=df_2[["const", "total_price", "shipping_costs", "actual_delivery_days"]]).fit()

In [43]:
lr_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     4112.
Date:                Tue, 04 Feb 2020   Prob (F-statistic):               0.00
Time:                        15:19:57   Log-Likelihood:            -1.5600e+05
No. Observations:               96470   AIC:                         3.120e+05
Df Residuals:                   96466   BIC:                         3.120e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.7518      0.007    647.042      0.000       4.737       4.766
total_price          -2.272e-05   2.06e-05     -1.103      0.270   -6.31e-05    1.77e-05
shipping_costs          -0.0021      0.000    -10.195      0.000      -0.002      -0.002
actual_delivery_days    -0.0446      0.000   -106.948      0.000      -0.045      -0.044
==============================================================================
Omnibus:                    19009.364   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33563.681
Skew:                          -1.278   Prob(JB):                         0.00
Kurtosis:                       4.348   Cond. No.                         470.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

With every additional day customers are waiting, the review score decreases by 0.0446, holding other variables constant (cp: ceteris paribus)

### Interpreting $R^2$:

11.3% of the variation is explained by the model

In [47]:
df = df.dropna()

In [48]:
sm.OLS(df["review_score"],
       df[["actual_delivery_days"]].assign(const=1)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_score   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                 1.218e+04
Date:                Tue, 04 Feb 2020   Prob (F-statistic):               0.00
Time:                        15:28:58   Log-Likelihood:            -1.5607e+05
No. Observations:               96470   AIC:                         3.121e+05
Df Residuals:                   96468   BIC:                         3.122e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
actual_delivery_days    -0.0454      0.000   -110.372      0.000      -0.046      -0.045
const                    4.7117      0.006    728.573      0.000       4.699       4.724
==============================================================================
Omnibus:                    19232.713   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34268.778
Skew:                          -1.285   Prob(JB):                         0.00
Kurtosis:                       4.385   Cond. No.                         26.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Lab: Practice!!! Play around with more sets of variables.

Before you run the model, write down what you think the result should look like.